# Coefficients & Residuals

In [97]:
import os
import typing as tp

In [98]:
import numpy as np
import pandas as pd
from numpy.typing import NDArray

In [99]:
UInt8s: tp.TypeAlias = NDArray[np.uint8]
Int64s: tp.TypeAlias = NDArray[np.int64]
Floats: tp.TypeAlias = NDArray[np.float64]

## 0. Meta

In [100]:
MIN: int = -5000
MAX: int = 9999
BPL: int = 12

In [101]:
here: str = os.path.abspath('')
csv_dir: str = os.path.join(here, 'build')
lu_path: str = os.path.join(csv_dir, 'lunar.csv')
so_path: str = os.path.join(csv_dir, 'solar.csv')

In [102]:
def int_frac(x: float) -> tuple[int, float]:
    '''
    Splits a float to integral and fractional parts.

    Args:
        x (float): input floating-point value
    Returns:
        int: the maximal integer `i <= x`
        float: `x - i` in [0, 1)
    '''

    xi, xf = divmod(x, 1)
    return int(xi), xf

## 1. Lunar

### 1.0. Data

In [103]:
lu_df: pd.DataFrame = pd.read_csv(lu_path)
lu_va: pd.Series = lu_df['nian'].between(MIN, MAX)
cyue: pd.Series = lu_df['cyue']
lu_df['uday'] = (lu_df['usec'] + 28800) // 86400
uday: pd.Series = lu_df['uday']
lu_df.head()

,cyue,nian,ryue,usec,uday
0,-86209,-5001,22,-219954231530,-2545767
1,-86208,-5001,24,-219951691034,-2545737
2,-86207,-5000,2,-219949157263,-2545708
3,-86206,-5000,4,-219946626852,-2545679
4,-86205,-5000,6,-219944096614,-2545649


### 1.1. N2G: (`nian`, `ryue`) to `cyue` to uday

#### 1.1.1. `nian` to `run`

In [104]:
nr_ryue: pd.Series = lu_df['ryue']
nr_rn: pd.Series = (lu_df['ryue'] & 1).astype(np.bool)
nr_nian: pd.Series = lu_df['nian'][nr_rn]
nr_ry: pd.Series = nr_ryue[nr_rn] // 2
nr_dc: dict[int, int] = dict(zip(nr_nian, nr_ry))
print(f'{len(nr_dc)} runs')

5708 runs


In [105]:
NR_BIT, NR_IPB = 4, 2
NR_IPL: int = NR_IPB * BPL
nr_cnt_va: int = (MAX - MIN + 1) + (MIN - MAX - 1) % NR_IPL
nr_ry_bt: list[int] = []
for i in range(0, nr_cnt_va, NR_IPB):
    bt: int = 0
    for j in range(NR_IPB):
        n: int = MIN + i + j
        bt |= nr_dc.get(n, 13) << (j * NR_BIT)
    nr_ry_bt.append(bt)
nr_ry_np: UInt8s = np.array(nr_ry_bt, dtype=np.uint8)
nr_ry_np = nr_ry_np.reshape((-1, BPL))
nr_ry_ln: list[list[int]] = nr_ry_np.tolist()
print(f'{len(nr_ry_bt)} bytes')

7500 bytes


#### 1.1.2. `nian` to `cyue`: linear (1-deg) regression

In [106]:
ny_va: pd.Series = lu_df['ryue'] == 2
ny_nian: pd.Series = lu_df['nian'][ny_va]
ny_expl: pd.Series = ny_nian - MIN
ny_dp01: pd.Series = lu_df['cyue'][ny_va]
ny_coef: Floats = np.polyfit(x=ny_expl, y=ny_dp01, deg=1)
ny_c1, ny_c0 = tp.cast(list[float], ny_coef.tolist())
print(f'cyue = {ny_c1} * (nian - {MIN}) + {ny_c0}')

cyue = 12.368273905932467 * (nian - -5000) + -86206.92050499133


In [107]:
ny_c1i, ny_c1f = int_frac(ny_c1)
ny_c0i, ny_c0f = int_frac(ny_c0)
print(ny_c1i, ny_c1f, ny_c0i, ny_c0f)

12 0.3682739059324671 -86207 0.07949500867107417


In [108]:
NY_BIT, NY_IPB = 1, 8
for ny_bits in range(32):
    ny_c0ib: int = ny_c0i
    ny_c1bi, _ = int_frac(ny_c1f * (1 << ny_bits))
    ny_c0bi, _ = int_frac(ny_c0f * (1 << ny_bits))
    ny_pred: pd.Series = ny_c1i * ny_expl + ny_c0i
    ny_bias: pd.Series = ny_c1bi * ny_expl + ny_c0bi
    ny_pred += ny_bias // (1 << ny_bits)
    ny_resd: pd.Series = ny_dp01 - ny_pred
    if ny_resd.max() - ny_resd.min() < (1 << NY_BIT):
        ny_c0ib += ny_resd.min() - ny_c1i * MIN
        ny_c0bi -= ny_c1bi * MIN
        ny_resd -= ny_resd.min()
        print(f'cyue = {ny_c1i} * nian + {ny_c0ib}', end=' + ')
        print(f'({ny_c1bi} * nian + {ny_c0bi} >> {ny_bits})')
        print(f'resd in {range(ny_resd.min(), ny_resd.max() + 1)}')
        break
else:
    raise ValueError('bad idea, residuals too wide')

cyue = 12 * nian + -26207 + (12067 * nian + 60337604 >> 15)
resd in range(0, 2)


In [109]:
NY_IPL: int = NY_IPB * BPL
ny_va: pd.Series = ny_nian.between(MIN, MAX)
ny_cnt_va: int = ny_va.sum() + 1 + (-(ny_va.sum() + 1) % NY_IPL)
ny_resd_va: pd.Series = ny_resd[ny_nian >= MIN].iloc[:ny_cnt_va]
ny_resd_bt: list[int] = []
for i in range(0, len(ny_resd_va), NY_IPB):
    bt: int = 0
    for j, r in enumerate(ny_resd_va.iloc[i:i+NY_IPB]):
        bt |= r << (j * NY_BIT)
    ny_resd_bt.append(bt)
ny_resd_np: UInt8s = np.array(ny_resd_bt, dtype=np.uint8)
ny_resd_np = ny_resd_np.reshape((-1, BPL))
ny_resd_ln: list[list[int]] = ny_resd_np.tolist()
print(f'{len(ny_resd_bt)} bytes')

1884 bytes


#### 1.1.3. `cyue` to uday: linear (1-deg) regression

In [110]:
yd_coef: Floats = np.polyfit(x=cyue, y=uday, deg=1)
yd_c1, yd_c0 = tp.cast(list[float], yd_coef.tolist())
print(f'uday = {yd_c1} * cyue + {yd_c0}')

uday = 29.53058513341781 * cyue + 35.7110938177826


In [111]:
yd_c1i, yd_c1f = int_frac(yd_c1)
yd_c0i, yd_c0f = int_frac(yd_c0)
print(yd_c1i, yd_c1f, yd_c0i, yd_c0f)

29 0.5305851334178087 35 0.7110938177825972


In [112]:
YD_BIT, YD_IPB = 2, 4
for yd_bits in range(64):
    yd_c1bi, _ = int_frac(yd_c1f * (1 << yd_bits))
    yd_c0bi, _ = int_frac(yd_c0f * (1 << yd_bits))
    yd_pred: pd.Series = yd_c1i * cyue + yd_c0i
    yd_bias: pd.Series = yd_c1bi * cyue + yd_c0bi
    yd_pred += yd_bias // (1 << yd_bits)
    yd_resd: pd.Series = uday - yd_pred
    if yd_resd.max() - yd_resd.min() < 1 << YD_BIT:
        yd_c0i += yd_resd.min()
        yd_resd -= yd_resd.min()
        print(f'uday = {yd_c1i} * cyue + {yd_c0i}', end=' + ')
        print(f'({yd_c1bi} * cyue + {yd_c0bi} >> {yd_bits})')
        print(f'resd in {range(yd_resd.min(), yd_resd.max() + 1)}')
        break
else:
    raise ValueError('bad idea, residuals too wide')

uday = 29 * cyue + 34 + (278179 * cyue + 372817 >> 19)
resd in range(0, 4)


In [113]:
YD_IPL: int = YD_IPB * BPL
yd_cnt_va: int = lu_va.sum() + 1 + (-(lu_va.sum() + 1) % YD_IPL)
yd_va: pd.Series = lu_df['nian'] >= MIN
yd_resd_va: pd.Series = yd_resd[yd_va].iloc[:yd_cnt_va]
yd_resd_bt: list[int] = []
for i in range(0, len(yd_resd_va), YD_IPB):
    bt: int = 0
    for j, r in enumerate(yd_resd_va.iloc[i:i+YD_IPB]):
        bt |= r << (j * YD_BIT)
    yd_resd_bt.append(bt)
yd_resd_np: UInt8s = np.array(yd_resd_bt, dtype=np.uint8)
yd_resd_np = yd_resd_np.reshape((-1, BPL))
yd_resd_ln: list[list[int]] = yd_resd_np.tolist()
print(f'{len(yd_resd_bt)} bytes')

46392 bytes


### 1.2. G2N: uday to `cyue` to (`nian`, `ryue`)

#### 1.2.1. uday to `cyue`: linear (1-deg) regression

In [114]:
dy_coef: Floats = np.polyfit(x=uday, y=cyue, deg=1)
dy_c1, dy_c0 = tp.cast(list[float], dy_coef.tolist())
dy_c0 += 0.5 # it is midpoint that is to be predicted
print(f'cyue = {dy_c1} * cday + {dy_c0}')

cyue = 0.03386319625845294 * cday + -0.7092917774042613


In [115]:
_, dy_c1f = int_frac(dy_c1)
dy_c0i, dy_c0f = int_frac(dy_c0)
print(dy_c1f, dy_c0i, dy_c0f)

0.03386319625845294 -1 0.2907082225957387


In [116]:
for dy_bits in range(64):
    dy_c1bi, _ = int_frac(dy_c1f * (1 << dy_bits))
    dy_c0bi, _ = int_frac(dy_c0f * (1 << dy_bits))
    dy_bias: pd.Series = dy_c1bi * uday + dy_c0bi
    dy_pred: pd.Series = dy_c0i + dy_bias // (1 << dy_bits)
    if np.all(dy_pred == cyue):
        print(f'cyue = {dy_c0i}', end=' + ')
        print(f'({dy_c1bi} * uday + {dy_c0bi} >> {dy_bits})')
        break
else:
    raise ValueError('bad idea, residuals too wide')

cyue = -1 + (8877 * uday + 76207 >> 18)


#### 1.2.2. `cyue` to `nian`: linear (1-deg) regression

In [117]:
yn_cyue: pd.Series = lu_df['cyue'][lu_df['ryue'] == 2]
yn_nian: pd.Series = lu_df['nian'][lu_df['ryue'] == 2]
yn_coef: Floats = np.polyfit(x=yn_cyue, y=yn_nian, deg=1)
yn_c1, yn_c0 = tp.cast(list[float], yn_coef.tolist())
yn_c0 += 0.5 # similarly, midpoints are considered
print(f'nian = {yn_c1} * cyue + {yn_c0}')

nian = 0.0808520257214669 * cyue + 1970.5041542503538


In [118]:
_, yn_c1f = int_frac(yn_c1)
yn_c0i, yn_c0f = int_frac(yn_c0)
print(yn_c1f, yn_c0i, yn_c0f)

0.0808520257214669 1970 0.5041542503538494


In [119]:
for yn_bits in range(64):
    yn_c1bi, _ = int_frac(yn_c1f * (1 << yn_bits))
    yn_c0bi, _ = int_frac(yn_c0f * (1 << yn_bits))
    yn_bias: pd.Series = yn_c1bi * yn_cyue + yn_c0bi
    yn_pred: pd.Series = yn_c0i + yn_bias // (1 << yn_bits)
    if np.all(yn_pred == yn_nian):
        print(f'nian = {yn_c0i}', end=' + ')
        print(f'({yn_c1bi} * cyue + {yn_c0bi} >> {yn_bits})')
        break
else:
    raise ValueError('bad idea, residuals too wide')

nian = 1970 + (10597 * cyue + 66080 >> 17)
